In [29]:
import pandas as pd
import json
from tqdm import tqdm
import time
import requests

import shinra_util as util

In [25]:
entry_json = util.read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

wikidata_ids = {entry['index']['_id']: entry['wikibase_item'] for entry in entry_json if entry.get('wikibase_item')}

with open("../data/pageID2wikidataID.json", 'w') as f:
    json.dump(wikidata_ids, f)

In [26]:
len(wikidata_ids)

5456

In [32]:
with open("../data/compound_wikidata.jsonl", 'a') as f:
    for page_id, wd_id in tqdm(wikidata_ids.items()):
        i = 0
        while True:
            if i > 5: break
            try:
                i += 1
                resp = requests.get(f'https://www.wikidata.org/wiki/Special:EntityData/{wd_id}.json')
                break
            except requests.HTTPError as e:
                print("HTTP error.")
                break
            except requests.ConnectionError as e:
                print("Connection error.")
                break
            except requests.ConnectTimeout as t:
                print("Connection timeout.\nRetry...")
                time.sleep(10)
                continue

        if resp is None or resp.status_code == 404:
            continue

        page_data = resp.json()
        json.dump(page_data, f)
        f.write('\n')

100%|██████████| 5456/5456 [1:03:28<00:00,  1.43it/s]
